In [300]:
#Подключаем библиотеки
import numpy as np
import pandas as pd
#Фиксируем счетчик для успешного повторения результата
np.random.seed(1)

In [301]:
#Настройки "игры"
left = 1                                          #Минимальное число
right = 100                                       #Максимальное число
N_try = 2500                                      #Количество попыток
#Прочие операции
right += 1

In [302]:
#Предсказать число (версия 1)
def predict_number_v1(number, Info=True):
    """
    Алгоритм предсказания числа
    number - число, которое необходимо предсказать
    Info - вывод информации в процессе поиска, если правда
    """
    count = 0                                     #Счетчик числа попыток
    predict = np.random.randint(left,right)       #Случайное предполагаемое число
    while True:                                   #Бесконечный цикл пока не угадаем число
        count += 1                                #Увеличиваем число попыток
        if predict > number:                      #Если предполагаемое число больше загаданного
            predict -= 1
        elif predict < number:                    #Если предполагаемое число меньше загаданного
            predict += 1
        else:                                     #Если предполагаемое число равно загаданному
            return count
        #Вывод информации
        if Info:
            print('Попытка {}: Загаданное число: {}. Предсказанное число: {}. Шаг: {}.'.format(count, number, predict, step))

#Предсказать число (версия 2)
def predict_number_v2(number, step=26, k_step=0.53, Info=True):
    """
    Алгоритм предсказания числа
    number - число, которое необходимо предсказать
    step - шаг, с которым алгоритм приближает предполагаемое число к загаданному
    k_step - коэффициент уменьшения шага step
    Info - вывод информации в процессе поиска, если правда
    """
    count = 0                                     #Счетчик числа попыток
    sign_count = [0, 0]
    predict = np.random.randint(left,right)       #Случайное предполагаемое число
    while True:                                   #Бесконечный цикл пока не угадаем число
        count += 1                                #Увеличиваем число попыток
        if sign_count[0] != sign_count[1]:        #Если знак отклонения отличается
            step = int(step * k_step)             #Уменьшаем шаг и приводим его к целому числу
            if step < 1:
                k_step = 1
                step = 1
        sign_count[0] = sign_count[1]             #Сохранение знака отклонения с предыдущего шага
        if predict > number:                      #Если предполагаемое число больше загаданного
            sign_count[1] = 1                     #Знак отклонения положителен
            predict -= step
        elif predict < number:                    #Если предполагаемое число меньше загаданного
            sign_count[1] = -1                    #Знак отклонения отрицателен
            predict += step
        else:                                     #Если предполагаемое число равно загаданному
            return count
        #Вывод информации
        if Info:
            print('Попытка {}: Загаданное число: {}. Предсказанное число: {}. Шаг: {}.'.format(count, number, predict, step))
        
#Загадать и предсказать число
def guess_and_predict(algo, Info=True):
    """
    Функция для загадывания и предсказывания числа
    algo - алгоритм, по которому ведется предсказания числа
    Info - вывод информации в процессе поиска, если правда
    """
    number = np.random.randint(left,right)        #Загадываем число
    count = algo(number, Info=Info)                 #Предсказываем число за число попыток count
    #Вывод результата
    if Info:
        print("Вы угадали число {} за {} попыток.".format(number,count))
    return count

In [303]:
'''
Тест второго алгоритма на оптимальные его параметры
'''
#Задаем массив с варьируемыми параметрами
mas_step = [[i, j*0.01] for i in range(20,41,1) for j in range (30,55,1)]
#Создаем пустой список для числа попыток
res = []
#Запускаем цикл
for step, k_step in mas_step:
    #Создаем промежуточную функцию
    def sub_func(number,Info=False):
        return predict_number_v2(number,step=step,k_step=k_step,Info=False)
    #Сохраняем число попыток при заданных параметрах алгоритма и проводим вычисления N_try раз
    mas_count = []
    for i in range(N_try):
        mas_count += [guess_and_predict(sub_func,Info=False)]
    #Вычисляем среднее и сохраняем в список
    res += [[step, k_step, np.mean(mas_count)]]
#Переводим список в DataFrame
res = pd.DataFrame(res)
#Определяем индекс элемента, имеющего минимальное число попыток
index_min = res.idxmin()[2]
#Сохраняем оптимальные параметры алгоритма, при которых число попыток минимально
mas_opt = res.iloc[index_min,0:2]
#Вывод результата
print('Оптимальные параметры: Шаг: {}. К-т уменьшения шага: {}.'.format(mas_opt[0],mas_opt[1]))

Оптимальные параметры: Шаг: 26.0. К-т уменьшения шага: 0.53.


In [304]:
'''
Сравнение двух алгоритмов
'''
#Создаем пустой список для хранения числа попыток
res = []
#Запускаем цикл
res = [[guess_and_predict(predict_number_v1,Info=False), guess_and_predict(predict_number_v2,Info=False)] for i in range(N_try)]
#Создаем DataFrame для хранения данных
res = pd.DataFrame(res, columns=['count v1','count v2'])
#Среднее значение
res_mean = res.mean()
#Вывод результата
print('Версия алгоритма 1: Среднее значение числа попыток: {}.'.format(res_mean[0]))
print('Версия алгоритма 2: Среднее значение числа попыток: {}.'.format(res_mean[1]))

Версия алгоритма 1: Среднее значение числа попыток: 34.6632.
Версия алгоритма 2: Среднее значение числа попыток: 7.5112.


In [305]:
'''
Результат
'''
count = guess_and_predict(predict_number_v2)

Попытка 1: Загаданное число: 76. Предсказанное число: 83. Шаг: 26.
Попытка 2: Загаданное число: 76. Предсказанное число: 70. Шаг: 13.
Попытка 3: Загаданное число: 76. Предсказанное число: 76. Шаг: 6.
Вы угадали число 76 за 4 попыток.
